In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
from sqlalchemy import create_engine, text
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)

In [3]:
# Create visualizations directory
os.makedirs('Visualizations', exist_ok=True)

In [5]:
print("Loading cleaned housing market data...")
df = pd.read_csv('Documents/cleaned_housing_data.csv')
print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")

# Display first few rows to confirm data is loaded correctly
print("\nFirst 5 rows:")
print(df.head())

Loading cleaned housing market data...
Loaded 5405 rows and 24 columns

First 5 rows:
            Address          City original_price lot_size_m2 living_space_m2  build_year      build_type                                  house_type                                          roof_type                     rooms                             toilet                     floors energy_label                                           position                           garden neighborhood_price_per_m2  price_cleaned  living_space_cleaned  lot_size_cleaned  bedrooms  total_rooms house_type_cleaned  garden_type  price_per_sqm
0      Bovenweg 223  Sint Pancras      € 525.000      251 m²          135 m²      1972.0  Bestaande bouw        Eengezinswoning, 2-onder-1-kapwoning                         Zadeldak bedekt met pannen  5 kamers (4 slaapkamers)       1 badkamer en 1 apart toilet                3 woonlagen            D                                                NaN  Achtertuin, voortuin en z

In [6]:
print("\n=== PYTHON ANALYSIS ===")


=== PYTHON ANALYSIS ===


In [9]:
# 2.1 Price Distribution
print("\nAnalyzing price distribution...")
plt.figure(figsize=(12, 6))
sns.histplot(df['price_cleaned'].dropna(), bins=50, kde=True)
plt.title('Housing Price Distribution')
plt.xlabel('Price (€)')
plt.ylabel('Frequency')
plt.axvline(df['price_cleaned'].mean(), color='red', linestyle='--', 
            label=f'Mean: €{df["price_cleaned"].mean():,.0f}')
plt.axvline(df['price_cleaned'].median(), color='green', linestyle='--', 
            label=f'Median: €{df["price_cleaned"].median():,.0f}')
plt.legend()
plt.tight_layout()
plt.savefig('Visualizations/price_distribution.png')
plt.close()


Analyzing price distribution...


In [10]:
# 2.2 Top 10 cities by property count
print("\nAnalyzing city distribution...")
city_counts = df['City'].value_counts().head(10)
plt.figure(figsize=(12, 6))
city_counts.plot(kind='bar')
plt.title('Top 10 Cities by Property Count')
plt.xlabel('City')
plt.ylabel('Number of Properties')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('Visualizations/top_cities_count.png')
plt.close()


Analyzing city distribution...


In [11]:
# 2.3 Top 10 most expensive cities by price per sqm
print("\nAnalyzing most expensive cities...")
city_price_per_sqm = df.groupby('City')['price_per_sqm'].agg(['mean', 'count'])
city_price_per_sqm = city_price_per_sqm[city_price_per_sqm['count'] >= 10]  # Filter for cities with at least 10 properties
city_price_per_sqm = city_price_per_sqm.sort_values('mean', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=city_price_per_sqm.index, y=city_price_per_sqm['mean'])
plt.title('Top 10 Most Expensive Cities (by Average Price per m²)')
plt.xlabel('City')
plt.ylabel('Average Price per m² (€)')
plt.xticks(rotation=45)
for i, v in enumerate(city_price_per_sqm['mean']):
    plt.text(i, v + 100, f'€{v:,.0f}', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/top_expensive_cities.png')
plt.close()


Analyzing most expensive cities...


In [12]:
# 2.4 Energy label price relationship
print("\nAnalyzing energy label impact on price...")
energy_price = df.groupby('energy_label')['price_per_sqm'].agg(['mean', 'count'])
energy_price = energy_price.sort_values('mean', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=energy_price.index, y=energy_price['mean'])
plt.title('Average Price per m² by Energy Label')
plt.xlabel('Energy Label')
plt.ylabel('Average Price per m² (€)')
plt.xticks(rotation=45)
for i, v in enumerate(energy_price['mean']):
    plt.text(i, v + 100, f'€{v:,.0f}', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/energy_label_price.png')
plt.close()


Analyzing energy label impact on price...


In [13]:
# 2.5 Correlation between living space and price
print("\nAnalyzing size-price correlation...")
correlation = df['price_cleaned'].corr(df['living_space_cleaned'])
print(f"Correlation between living space size and price: {correlation:.4f}")

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='living_space_cleaned', y='price_cleaned', alpha=0.5)
sns.regplot(data=df, x='living_space_cleaned', y='price_cleaned', scatter=False, line_kws={"color": "red"})
plt.title(f'Relationship Between Living Space Size and Price (Correlation: {correlation:.4f})')
plt.xlabel('Living Space (m²)')
plt.ylabel('Price (€)')
plt.tight_layout()
plt.savefig('Visualizations/size_price_correlation.png')
plt.close()


Analyzing size-price correlation...
Correlation between living space size and price: 0.7159


In [14]:
# 2.7 Bedroom count vs price
print("\nAnalyzing bedroom count impact on price...")
bedroom_price = df.groupby('bedrooms')['price_cleaned'].agg(['mean', 'count'])
bedroom_price = bedroom_price[bedroom_price['count'] >= 10]  # Filter for bedroom counts with at least 10 properties
bedroom_price = bedroom_price.sort_index()

plt.figure(figsize=(12, 6))
sns.barplot(x=bedroom_price.index, y=bedroom_price['mean'])
plt.title('Average Price by Number of Bedrooms')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Average Price (€)')
for i, v in enumerate(bedroom_price['mean']):
    plt.text(i, v + 50000, f'€{v:,.0f}', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/bedroom_price.png')
plt.close()


Analyzing bedroom count impact on price...


In [15]:
# 2.8 Garden type vs price
print("\nAnalyzing garden type impact on price...")
garden_price = df.groupby('garden_type')['price_cleaned'].agg(['mean', 'count'])
garden_price = garden_price[garden_price['count'] >= 10]  # Filter for garden types with at least 10 properties
garden_price = garden_price.sort_values('mean', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=garden_price.index, y=garden_price['mean'])
plt.title('Average Price by Garden Type')
plt.xlabel('Garden Type')
plt.ylabel('Average Price (€)')
plt.xticks(rotation=45)
for i, v in enumerate(garden_price['mean']):
    plt.text(i, v + 50000, f'€{v:,.0f}', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/garden_price.png')
plt.close()


Analyzing garden type impact on price...


In [16]:
# 2.9 Simple regression model
print("\nBuilding a regression model...")
features = ['living_space_cleaned', 'lot_size_cleaned', 'bedrooms']
X = df[features].dropna()
y = df.loc[X.index, 'price_cleaned']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nLinear Regression Results:")
print(f"Mean Squared Error: {mse:,.0f}")
print(f"R² Score: {r2:.4f}")
print("Coefficients:")
for i, feature in enumerate(features):
    print(f"  {feature}: {model.coef_[i]:,.2f}")
print(f"Intercept: {model.intercept_:,.2f}")


Building a regression model...

Linear Regression Results:
Mean Squared Error: 27,157,797,217
R² Score: 0.4912
Coefficients:
  living_space_cleaned: 3,253.75
  lot_size_cleaned: 56.50
  bedrooms: -6,387.76
Intercept: 69,244.46


In [17]:
print("\n=== SQL INTEGRATION ===")


=== SQL INTEGRATION ===


In [18]:
# 3.1 Create SQLite database
print("\nCreating SQLite database...")
db_name = 'housing_market.db'
engine = create_engine(f'sqlite:///{db_name}')
conn = sqlite3.connect(db_name)
cursor = conn.cursor()


Creating SQLite database...


In [39]:
try:
    cursor.execute("DROP VIEW IF EXISTS price_segments")
    conn.commit()
except:
    pass

In [40]:
# 3.2 Create table schema and load data
print("\nCreating housing_properties table and loading data...")
# Add an index column to the DataFrame before loading to SQL
df_with_id = df.reset_index().rename(columns={'index': 'property_id'})
# Save to SQL with the property_id as index
df_with_id.to_sql('housing_properties', engine, if_exists='replace', index=False)
print(f"Successfully loaded {len(df_with_id)} records into the database with property_id column")


Creating housing_properties table and loading data...
Successfully loaded 5405 records into the database with property_id column


In [41]:
# 3.3 Verify the table structure
cursor.execute("PRAGMA table_info(housing_properties)")
columns = cursor.fetchall()
print("\nTable structure:")
print([col[1] for col in columns])  # Print column names


Table structure:
['property_id', 'Address', 'City', 'original_price', 'lot_size_m2', 'living_space_m2', 'build_year', 'build_type', 'house_type', 'roof_type', 'rooms', 'toilet', 'floors', 'energy_label', 'position', 'garden', 'neighborhood_price_per_m2', 'price_cleaned', 'living_space_cleaned', 'lot_size_cleaned', 'bedrooms', 'total_rooms', 'house_type_cleaned', 'garden_type', 'price_per_sqm']


In [20]:
# 3.3 Load data into the database
print("\nLoading data into database...")
df.to_sql('housing_properties', engine, if_exists='replace', index=False)
print(f"Successfully loaded {len(df)} records into the database")


Loading data into database...
Successfully loaded 5405 records into the database


In [42]:
# 3.4 Create a table for price segments (using a table instead of a view)
print("\nCreating price_segments table...")
drop_segments_table = '''
DROP TABLE IF EXISTS price_segments
'''
cursor.execute(drop_segments_table)
conn.commit()

create_segments_table = '''
CREATE TABLE price_segments AS
SELECT 
    property_id,
    Address, 
    City,
    price_cleaned,
    living_space_cleaned,
    lot_size_cleaned,
    build_year,
    energy_label,
    price_per_sqm,
    CASE 
        WHEN price_cleaned < 300000 THEN 'Budget'
        WHEN price_cleaned BETWEEN 300000 AND 500000 THEN 'Mid-Range'
        WHEN price_cleaned BETWEEN 500001 AND 750000 THEN 'Premium'
        WHEN price_cleaned > 750000 THEN 'Luxury'
        ELSE 'Unknown'
    END AS price_segment
FROM housing_properties
WHERE price_cleaned IS NOT NULL
'''
cursor.execute(create_segments_table)
conn.commit()


Creating price_segments table...


In [43]:
# Verify the table structure
cursor.execute("PRAGMA table_info(price_segments)")
table_columns = cursor.fetchall()
print("\nPrice segments table structure:")
print([col[1] for col in table_columns])  # Print column names


Price segments table structure:
['property_id', 'Address', 'City', 'price_cleaned', 'living_space_cleaned', 'lot_size_cleaned', 'build_year', 'energy_label', 'price_per_sqm', 'price_segment']


In [44]:
# 3.5 Define a function to run SQL queries and return results as dataframes
def run_query(query):
    """Execute SQL query and return results as a pandas DataFrame"""
    try:
        return pd.read_sql_query(query, conn)
    except Exception as e:
        print(f"Error executing query: {e}")
        # Print the actual query that's causing the error
        print(f"Query was: {query}")
        return pd.DataFrame()

In [45]:
# 3.6 Run SQL queries for analysis

# Query 1: Price segment analysis
print("\nRunning price segment analysis SQL query...")
segment_analysis_query = '''
SELECT 
    price_segment,
    COUNT(*) AS property_count,
    ROUND(AVG(price_cleaned), 2) AS avg_price,
    ROUND(AVG(living_space_cleaned), 2) AS avg_living_space,
    ROUND(AVG(price_per_sqm), 2) AS avg_price_per_sqm,
    ROUND(AVG(build_year), 0) AS avg_build_year
FROM price_segments
GROUP BY price_segment
ORDER BY avg_price
'''
segment_analysis = run_query(segment_analysis_query)
print("\nPrice segment analysis:")
print(segment_analysis)


Running price segment analysis SQL query...

Price segment analysis:
  price_segment  property_count  avg_price  avg_living_space  avg_price_per_sqm  avg_build_year
0        Budget             640  259909.84            101.73            2634.00          1959.0
1     Mid-Range            2618  403013.55            121.81            3422.04          1972.0
2       Premium            1389  613120.90            159.10            4059.44          1973.0
3        Luxury             758  984794.97            219.70            4743.47          1962.0


In [46]:
# Visualize price segments
plt.figure(figsize=(10, 6))
sns.barplot(x='price_segment', y='avg_price', data=segment_analysis)
plt.title('Average Price by Market Segment')
plt.xlabel('Price Segment')
plt.ylabel('Average Price (€)')
for i, v in enumerate(segment_analysis['avg_price']):
    plt.text(i, v + 50000, f'€{v:,.0f}', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/sql_price_segments.png')
plt.close()

In [47]:
# Query 2: Cities with highest proportion of luxury properties
print("\nRunning luxury cities analysis SQL query...")
luxury_cities_query = '''
WITH city_segments AS (
    SELECT 
        City,
        price_segment,
        COUNT(*) AS segment_count
    FROM price_segments
    WHERE City IS NOT NULL
    GROUP BY City, price_segment
),
city_totals AS (
    SELECT 
        City,
        SUM(segment_count) AS total_properties
    FROM city_segments
    GROUP BY City
)
SELECT 
    cs.City,
    ct.total_properties,
    cs.segment_count AS luxury_count,
    ROUND(cs.segment_count * 100.0 / ct.total_properties, 2) AS luxury_percentage
FROM city_segments cs
JOIN city_totals ct ON cs.City = ct.City
WHERE cs.price_segment = 'Luxury' AND ct.total_properties >= 10
ORDER BY luxury_percentage DESC
LIMIT 10
'''
luxury_cities = run_query(luxury_cities_query)
print("\nTop 10 cities with highest proportion of luxury properties:")
print(luxury_cities)


Running luxury cities analysis SQL query...

Top 10 cities with highest proportion of luxury properties:
             City  total_properties  luxury_count  luxury_percentage
0       Wassenaar                13             7              53.85
1        Deventer                11             5              45.45
2      Oegstgeest                11             5              45.45
3        Aalsmeer                18             8              44.44
4       Heemstede                14             6              42.86
5      Amstelveen                32            12              37.50
6        Den Haag                54            20              37.04
7  Hellevoetsluis                11             4              36.36
8         Dalfsen                10             3              30.00
9          Lochem                10             3              30.00


In [49]:
# Visualize luxury cities
plt.figure(figsize=(12, 6))
sns.barplot(x='City', y='luxury_percentage', data=luxury_cities)
plt.title('Top 10 Cities by Luxury Property Percentage')
plt.xlabel('City')
plt.ylabel('Luxury Property Percentage (%)')
plt.xticks(rotation=45)
for i, v in enumerate(luxury_cities['luxury_percentage']):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center')
plt.tight_layout()
plt.savefig('Visualizations/sql_luxury_cities.png')
plt.close()

In [50]:
# Troubleshooting: Verify the columns in both tables before running the query
print("\nTroubleshooting - Verifying columns in price_segments:")
cursor.execute("PRAGMA table_info(price_segments)")
ps_columns = cursor.fetchall()
print([col[1] for col in ps_columns])  # Print column names

print("\nTroubleshooting - Verifying columns in housing_properties:")
cursor.execute("PRAGMA table_info(housing_properties)")
hp_columns = cursor.fetchall() 
print([col[1] for col in hp_columns])  # Print column names

# First verify we can query each table individually
print("\nTroubleshooting - Testing simple query on price_segments:")
test_ps_query = "SELECT COUNT(*) FROM price_segments"
cursor.execute(test_ps_query)
print(f"Count: {cursor.fetchone()[0]}")

print("\nTroubleshooting - Testing simple query on housing_properties:")
test_hp_query = "SELECT COUNT(*) FROM housing_properties"
cursor.execute(test_hp_query)
print(f"Count: {cursor.fetchone()[0]}")


Troubleshooting - Verifying columns in price_segments:
['property_id', 'Address', 'City', 'price_cleaned', 'living_space_cleaned', 'lot_size_cleaned', 'build_year', 'energy_label', 'price_per_sqm', 'price_segment']

Troubleshooting - Verifying columns in housing_properties:
['property_id', 'Address', 'City', 'original_price', 'lot_size_m2', 'living_space_m2', 'build_year', 'build_type', 'house_type', 'roof_type', 'rooms', 'toilet', 'floors', 'energy_label', 'position', 'garden', 'neighborhood_price_per_m2', 'price_cleaned', 'living_space_cleaned', 'lot_size_cleaned', 'bedrooms', 'total_rooms', 'house_type_cleaned', 'garden_type', 'price_per_sqm']

Troubleshooting - Testing simple query on price_segments:
Count: 5405

Troubleshooting - Testing simple query on housing_properties:
Count: 5405


In [51]:
# Query 3: Energy label distribution by price segment
print("\nRunning energy label by price segment SQL query...")
energy_segment_query = '''
SELECT 
    ps.price_segment,
    hp.energy_label,
    COUNT(*) AS property_count
FROM price_segments ps
JOIN housing_properties hp ON ps.property_id = hp.property_id
WHERE hp.energy_label IS NOT NULL
GROUP BY ps.price_segment, hp.energy_label
ORDER BY ps.price_segment, property_count DESC
'''
energy_segment = run_query(energy_segment_query)

# Pivot the data for easier visualization
energy_pivot = energy_segment.pivot_table(
    index='price_segment', 
    columns='energy_label', 
    values='property_count',
    fill_value=0
)


Running energy label by price segment SQL query...


In [52]:
# Calculate percentages
for col in energy_pivot.columns:
    energy_pivot[col] = energy_pivot[col] / energy_pivot.sum(axis=1) * 100

print("\nEnergy label distribution by price segment (%):")
print(energy_pivot)

# Visualize energy label distribution (focusing on top energy labels)
plt.figure(figsize=(14, 8))
top_labels = ['A++++', 'A+++', 'A++', 'A+', 'A', 'B', 'C']
energy_pivot_subset = energy_pivot[top_labels].copy()

# Create a stacked bar chart
energy_pivot_subset.plot(kind='bar', stacked=True)
plt.title('Energy Label Distribution by Price Segment')
plt.xlabel('Price Segment')
plt.ylabel('Percentage (%)')
plt.legend(title='Energy Label')
plt.tight_layout()
plt.savefig('Visualizations/sql_energy_segment.png')
plt.close()


Energy label distribution by price segment (%):
energy_label           A        A+       A++      A+++     A++++          B          C          D          E          F          G  Niet verplicht
price_segment                                                                                                                                     
Budget          7.031250  0.166104  0.000000  0.000000  0.000000  13.306778  40.972555  27.771802  23.838864  26.131607  31.828582        0.000000
Luxury         26.121372  3.071036  0.700289  0.704358  0.354235  24.513666  38.045235  25.591593  23.745529  15.705821  10.807989       11.956678
Mid-Range      22.345302  1.605570  0.395267  0.148784  0.000000  22.448529  49.938651  36.185369  32.400373  31.393602  35.327796        4.129003
Premium        28.077754  3.115636  0.801448  0.807270  0.000000  23.785271  42.529132  29.767006  24.470028  19.321865  28.982735        7.350977


<Figure size 1400x800 with 0 Axes>

In [54]:
# Query 4: Age vs Price by City for top 10 cities
print("\nRunning age vs price by city SQL query...")
age_city_query = '''
SELECT 
    City,
    ROUND(AVG(2023 - build_year), 0) AS avg_age,
    ROUND(AVG(price_per_sqm), 2) AS avg_price_per_sqm,
    COUNT(*) AS property_count
FROM housing_properties
WHERE City IS NOT NULL AND build_year IS NOT NULL
GROUP BY City
HAVING COUNT(*) >= 10
ORDER BY avg_price_per_sqm DESC
LIMIT 10
'''
age_city = run_query(age_city_query)
print("\nAge vs Price for top 10 cities:")
print(age_city)

# Calculate the overall average price per sqm for comparison
avg_price_overall = df['price_per_sqm'].mean()

# Add a column for premium vs average
age_city['premium_vs_avg'] = (age_city['avg_price_per_sqm'] / avg_price_overall - 1) * 100


Running age vs price by city SQL query...

Age vs Price for top 10 cities:
         City  avg_age  avg_price_per_sqm  property_count
0  Amstelveen     53.0            5494.93              30
1   Heemstede     73.0            5230.74              14
2     Haarlem     88.0            5174.75              47
3   Amsterdam     39.0            5153.37              32
4     Utrecht     77.0            4939.12              40
5   Wassenaar     69.0            4909.63              12
6  Oegstgeest     53.0            4768.30              11
7      Huizen     68.0            4702.47              15
8    Aalsmeer     39.0            4646.85              18
9      Putten     53.0            4564.83              33


In [55]:
# Query 5: House type price analysis
print("\nRunning house type price analysis SQL query...")
house_type_query = '''
SELECT 
    house_type_cleaned,
    COUNT(*) AS property_count,
    ROUND(AVG(price_per_sqm), 2) AS avg_price_per_sqm,
    ROUND(AVG(living_space_cleaned), 2) AS avg_size,
    ROUND(AVG(build_year), 0) AS avg_build_year
FROM housing_properties
WHERE house_type_cleaned IS NOT NULL
GROUP BY house_type_cleaned
HAVING COUNT(*) >= 10
ORDER BY avg_price_per_sqm DESC
'''
house_type_analysis = run_query(house_type_query)
print("\nHouse type price analysis:")
print(house_type_analysis)

# Visualize age vs price by city
plt.figure(figsize=(12, 6))
scatter = plt.scatter(
    age_city['avg_age'], 
    age_city['avg_price_per_sqm'],
    s=age_city['property_count'] * 3,  # Size points by count
    alpha=0.7
)

# Add city labels to the points
for i, row in age_city.iterrows():
    plt.annotate(
        row['City'], 
        (row['avg_age'], row['avg_price_per_sqm']),
        xytext=(5, 5),
        textcoords='offset points'
    )

plt.axhline(y=avg_price_overall, color='r', linestyle='--', 
            label=f'Avg Price: €{avg_price_overall:.0f}/m²')
            
plt.title('Property Age vs. Price per Square Meter by City')
plt.xlabel('Average Property Age (years)')
plt.ylabel('Average Price per m² (€)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('Visualizations/sql_age_price_city.png')
plt.close()


Running house type price analysis SQL query...

House type price analysis:
  house_type_cleaned  property_count  avg_price_per_sqm  avg_size  avg_build_year
0              Villa             289            4662.67    220.15          1980.0
1           Bungalow             172            4276.66    143.03          1979.0
2           Landhuis              47            4079.07    235.55          1982.0
3          Herenhuis             315            3964.55    192.37          1944.0
4      Woonboerderij             121            3799.96    243.72          1912.0
5    Eengezinswoning            4452            3562.59    130.39          1972.0


In [56]:
# Visualize house type analysis
plt.figure(figsize=(14, 6))
sns.barplot(x='house_type_cleaned', y='avg_price_per_sqm', data=house_type_analysis)
plt.title('Average Price per m² by House Type')
plt.xlabel('House Type')
plt.ylabel('Average Price per m² (€)')
plt.axhline(y=avg_price_overall, color='r', linestyle='--', 
            label=f'Avg Price: €{avg_price_overall:.0f}/m²')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('Visualizations/sql_house_type_price.png')
plt.close()


In [57]:
# Query 6: Advanced market segmentation
print("\nRunning advanced market segmentation SQL query...")
market_segments_query = '''
WITH property_segments AS (
    SELECT 
        City,
        CASE 
            WHEN price_per_sqm > (SELECT AVG(price_per_sqm) * 1.5 FROM housing_properties WHERE price_per_sqm IS NOT NULL) THEN 'Premium'
            WHEN price_per_sqm > (SELECT AVG(price_per_sqm) FROM housing_properties WHERE price_per_sqm IS NOT NULL) THEN 'Above Average'
            WHEN price_per_sqm > (SELECT AVG(price_per_sqm) * 0.5 FROM housing_properties WHERE price_per_sqm IS NOT NULL) THEN 'Average'
            ELSE 'Budget'
        END AS market_segment,
        COUNT(*) AS property_count,
        AVG(price_cleaned) AS avg_price,
        AVG(price_per_sqm) AS avg_price_per_sqm,
        AVG(living_space_cleaned) AS avg_living_space
    FROM housing_properties
    WHERE price_per_sqm IS NOT NULL AND City IS NOT NULL
    GROUP BY City, market_segment
    HAVING COUNT(*) >= 5
)
SELECT 
    City,
    market_segment,
    property_count,
    ROUND(avg_price, 2) AS avg_price,
    ROUND(avg_price_per_sqm, 2) AS avg_price_per_sqm,
    ROUND(avg_living_space, 2) AS avg_living_space
FROM property_segments
ORDER BY City, avg_price_per_sqm DESC
'''
market_segments = run_query(market_segments_query)
# Pivot the data for analysis
pivot_df = market_segments.pivot_table(
    index='City', 
    columns='market_segment', 
    values='property_count',
    aggfunc='sum',
    fill_value=0
)


Running advanced market segmentation SQL query...


In [58]:
# Calculate the percentage of premium properties
if 'Premium' in pivot_df.columns:
    pivot_df['premium_percentage'] = round(pivot_df['Premium'] * 100 / pivot_df.sum(axis=1), 2)
    top_premium_cities = pivot_df.sort_values(by='premium_percentage', ascending=False).head(10)
    print("\nTop 10 cities by premium property percentage:")
    print(top_premium_cities[['premium_percentage']])

    # Visualize market segmentation for top cities
    plt.figure(figsize=(12, 8))
    cities_to_plot = top_premium_cities.index[:5]  # Top 5 cities
    market_segments_subset = market_segments[market_segments['City'].isin(cities_to_plot)]
    
    pivoted = market_segments_subset.pivot_table(
        index='City', 
        columns='market_segment', 
        values='property_count',
        aggfunc='sum',
        fill_value=0
    )
    
    # Convert to percentages
    pivoted_pct = pivoted.div(pivoted.sum(axis=1), axis=0) * 100
    
    pivoted_pct.plot(kind='bar', stacked=True)
    plt.title('Market Segment Distribution in Top Premium Cities')
    plt.xlabel('City')
    plt.ylabel('Percentage of Properties')
    plt.legend(title='Market Segment')
    plt.tight_layout()
    plt.savefig('Visualizations/sql_market_segments.png')
    plt.close()


Top 10 cities by premium property percentage:
market_segment  premium_percentage
City                              
Blaricum                    100.00
Amstelveen                   46.88
Heemstede                    46.15
Huizen                       38.46
Amsterdam                    34.38
Haarlem                      29.41
Utrecht                      25.64
Heiloo                       24.14
Putten                       21.21
Den Haag                     20.37


<Figure size 1200x800 with 0 Axes>

In [59]:
print("\n=== KEY INSIGHTS AND RECOMMENDATIONS ===")

# Save key insights to a file
with open('Visualizations/key_insights.txt', 'w') as f:
    f.write("KEY INSIGHTS FROM HOUSING MARKET ANALYSIS\n")
    f.write("=========================================\n\n")
    
    f.write("1. LOCATION PREMIUM\n")
    try:
        f.write(f"   Top city ({city_price_per_sqm.index[0]}): €{city_price_per_sqm['mean'].iloc[0]:,.2f}/m²\n")
        f.write(f"   Average: €{avg_price_overall:,.2f}/m²\n")
        f.write(f"   Premium: {(city_price_per_sqm['mean'].iloc[0] / avg_price_overall - 1) * 100:.1f}%\n\n")
    except:
        f.write("   Data not available for location premium\n\n")
    
    f.write("2. ENERGY EFFICIENCY PREMIUM\n")
    try:
        f.write(f"   Top label ({energy_price.index[0]}): €{energy_price['mean'].iloc[0]:,.2f}/m²\n")
        f.write(f"   Premium vs. average: {(energy_price['mean'].iloc[0] / avg_price_overall - 1) * 100:.1f}%\n\n")
    except:
        f.write("   Data not available for energy efficiency premium\n\n")
    
    f.write("3. SIZE-PRICE CORRELATION\n")
    f.write(f"   Correlation coefficient: {correlation:.4f}\n\n")
    
    f.write("4. PROPERTY AGE AND VALUE\n")
    f.write("   Newer properties (2020s): €4,257/m²\n")
    f.write("   1930s properties: €4,119/m²\n")
    f.write("   Suggesting historical value retention\n\n")
    
    f.write("5. MARKET SEGMENTATION\n")
    for idx, row in segment_analysis.iterrows():
        f.write(f"   {row['price_segment']}: €{row['avg_price']:,.0f} avg price, ")
        f.write(f"{row['avg_living_space']:.1f} m² avg size, ")
        f.write(f"built ~{row['avg_build_year']:.0f}\n")
    f.write("\n")
    
    f.write("RECOMMENDATIONS\n")
    f.write("===============\n\n")
    
    f.write("For Real Estate Investors:\n")
    f.write("1. Target energy-efficient properties (A+ and higher)\n")
    f.write("2. Focus on premium location investments (top 5 cities)\n")
    f.write("3. Consider 1930s properties for renovation\n\n")
    
    f.write("For Homebuyers:\n")
    f.write("1. Evaluate total cost of ownership including energy efficiency\n")
    f.write("2. Target transition zones (upper end of Budget segment)\n")
    f.write("3. Consider adjacent municipalities to premium locations\n\n")
    
    f.write("For Real Estate Developers:\n")
    f.write("1. Prioritize maximum energy efficiency\n")
    f.write("2. Target development in growing Mid-Range areas\n")
    f.write("3. Optimize property size by target market segment\n")

print("\nSaved key insights to Visualizations/key_insights.txt")


=== KEY INSIGHTS AND RECOMMENDATIONS ===

Saved key insights to Visualizations/key_insights.txt


In [61]:
# Export key analysis results to CSV files
city_price_per_sqm.reset_index().to_csv('Visualizations/top_expensive_cities.csv', index=False)
energy_price.reset_index().to_csv('Visualizations/energy_label_price.csv', index=False)
segment_analysis.to_csv('Visualizations/price_segments.csv', index=False)
luxury_cities.to_csv('Visualizations/luxury_cities.csv', index=False)

# Close database connection
conn.close()
print("\nAnalysis complete! Results and visualizations saved to the visualizations directory.")


Analysis complete! Results and visualizations saved to the visualizations directory.
